# PRS map - Nagelkerke_vs_Tjur_vs_AUC

Yosuke Tanigawa


## library, functions, and constants

In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
source('paths.sh')
devtools::load_all(cud4_d)
devtools::load_all(dirname(dirname(snpnet_helper)))
source(snpnet_helper)


ℹ Loading cud4

ℹ Loading snpnet



In [3]:
traits_w_metrics_f %>%
fread() %>%
rename_with(
    function(x){str_replace(x, '#', '')}, starts_with("#")
) -> traits_w_metrics_df


eval_fullwDelta_f %>%
fread() %>%
rename_with(
    function(x){str_replace(x, '#', '')}, starts_with("#")
) -> eval_long_df


In [23]:
eval_long_df %>%
filter(
    WBtest_is_significant,
    split == "test",
    family == "binomial",
) %>%
select(trait, trait_name, trait_label, trait_category_plot, metric, model, value) %>%
pivot_wider(
    names_from = c("metric", "model"),
    values_from = c("value")
) -> eval_bin_wide_df


In [27]:
for(ext in c('png', 'pdf')){ggsave(
    file.path("plots", sprintf('Nagelkerke_vs_Tjur_vs_AUC.%s', ext)),
    gridExtra::arrangeGrob(
        # Nagelkerke's pseudo-R2
        eval_bin_wide_df %>%
        mutate(
            plot_label = if_else(rank(-auc_PRS) <= 5, trait_label, "")
        ) %>%
        ggplot(aes(x = NagelkerkeR2_PRS, y = auc_PRS, color=trait_category_plot, label = trait_label)) +
        geom_point() +
        theme_bw(base_size = 16) +
        labs(
            x = latex2exp::TeX("Nagelkerke's \\textit{R}$^2$ for PRS in white British"),
            y = "ROC-AUC for PRS in white British",
            color = 'Trait category'
        ) + 
        theme(legend.position = 'bottom') +
        guides(
          color = guide_legend(
            title = 'Trait category',
            override.aes = aes(label = "", alpha=1),
            ncol=2
          )
        ) +
        ggrepel::geom_text_repel(max.overlaps=30, force=20, mapping = aes(label = plot_label)),
        # Tjur's pseudo-R2
        eval_bin_wide_df %>%
        mutate(
            plot_label = if_else(rank(-auc_PRS) <= 5, trait_label, "")
        ) %>%
        ggplot(aes(x = TjurR2_PRS, y = auc_PRS, color=trait_category_plot, label = trait_label)) +
        geom_point() +
        theme_bw(base_size = 16) +
        labs(
            x = latex2exp::TeX("Tjur's \\textit{R}$^2$ for PRS in white British"),
            y = "ROC-AUC for PRS in white British",
            color = 'Trait category'
        ) + 
        theme(legend.position = 'bottom') +
        guides(
          color = guide_legend(
            title = 'Trait category',
            override.aes = aes(label = "", alpha=1),
            ncol=2
          )
        ) +
        ggrepel::geom_text_repel(max.overlaps=30, force=20, mapping = aes(label = plot_label)),
        ncol=2
    ),
    width=16, height=8
)}

